In [8]:
import pandas as pd
from pandas import json_normalize
import json
import re

In [4]:
# Cargar datos desde un archivo JSON
with open('./liga brasil/game_lineups.json', 'r') as file:
    data = json.load(file)

# Convertir la lista de diccionarios a un DataFrame
df = pd.DataFrame(data)

# Aplanar la estructura anidada del JSON para los datos de alineaciones de ambos equipos
game_lineups = []
for game in data:
    home_starting_lineup = game['home_club']['starting_lineup']
    home_substitutes = game['home_club']['substitutes']
    away_starting_lineup = game['away_club']['starting_lineup']
    away_substitutes = game['away_club']['substitutes']

    for player in home_starting_lineup + home_substitutes:
        player['club'] = 'home'
        player['formation'] = game['home_club']['formation']
        player['game_id'] = game['game_id']

    for player in away_starting_lineup + away_substitutes:
        player['club'] = 'away'
        player['formation'] = game['away_club']['formation']
        player['game_id'] = game['game_id']

    game_lineups.extend(home_starting_lineup)
    game_lineups.extend(home_substitutes)
    game_lineups.extend(away_starting_lineup)
    game_lineups.extend(away_substitutes)

# Convertir a DataFrame
df_flat = pd.DataFrame(game_lineups)

# Reemplazar columnas
df_flat.rename(columns={'number': 'player_number', 'href': 'player_href', 'name': 'player_name', 'team_captain': 'is_team_captain'}, inplace=True)

In [5]:
df_flat.head()

,player_number,player_href,player_name,is_team_captain,position,club,formation,game_id
0,\n 1\n,/bento/profil/spieler/691906,Bento,0,Goalkeeper,home,4-2-3-1,4316294
1,\n 15\n,/mateo-gamarra/profil/spieler/661245,Mateo Gamarra,0,Centre-Back,home,4-2-3-1,4316294
2,\n 4\n,/kaique-rocha/profil/spieler/633377,Kaique Rocha,0,Centre-Back,home,4-2-3-1,4316294
3,\n 37\n,/lucas-esquivel/profil/spieler/829969,Lucas Esquivel,0,Left-Back,home,4-2-3-1,4316294
4,\n 29\n,/leonardo-godoy/profil/spieler/369556,Leonardo Godoy,0,Right-Back,home,4-2-3-1,4316294


In [6]:
df_flat['player_number'] = df_flat['player_number'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [7]:
df_flat.head()

,player_number,player_href,player_name,is_team_captain,position,club,formation,game_id
0,1,/bento/profil/spieler/691906,Bento,0,Goalkeeper,home,4-2-3-1,4316294
1,15,/mateo-gamarra/profil/spieler/661245,Mateo Gamarra,0,Centre-Back,home,4-2-3-1,4316294
2,4,/kaique-rocha/profil/spieler/633377,Kaique Rocha,0,Centre-Back,home,4-2-3-1,4316294
3,37,/lucas-esquivel/profil/spieler/829969,Lucas Esquivel,0,Left-Back,home,4-2-3-1,4316294
4,29,/leonardo-godoy/profil/spieler/369556,Leonardo Godoy,0,Right-Back,home,4-2-3-1,4316294


In [10]:
# Función para extraer el número de la cadena
def extraer_id_player(cadena):
    match = re.search(r'/(\d+)$', cadena)
    if match:
        return int(match.group(1))
    else:
        return None

# Aplicar la función a cada fila de la columna
df_flat['player_id'] = df_flat['player_href'].apply(extraer_id_player)
df_flat.drop('player_href',axis=1,inplace=True)
df_flat.head()

,player_number,player_name,is_team_captain,position,club,formation,game_id,player_id
0,1,Bento,0,Goalkeeper,home,4-2-3-1,4316294,691906
1,15,Mateo Gamarra,0,Centre-Back,home,4-2-3-1,4316294,661245
2,4,Kaique Rocha,0,Centre-Back,home,4-2-3-1,4316294,633377
3,37,Lucas Esquivel,0,Left-Back,home,4-2-3-1,4316294,829969
4,29,Leonardo Godoy,0,Right-Back,home,4-2-3-1,4316294,369556


In [12]:
df_flat.to_csv('./liga brasil/game_lineups.csv',index=False)